In [ ]:
conda create --name simpletuner python=3.12 ipykernel -y 
conda activate simpletuner


# for scripy
apt-get update && apt-get install -y gfortran  
apt-get update && apt-get install -y \
  pkg-config \
  cmake \
  libopenblas-dev \
  liblapack-dev

# manually install torch 2.6
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu126


# patched to remove some old deps; then install reqs
pip install -r requirements.txt

In [ ]:
# fix triton
pip install --force-reinstall -v "triton==3.1.0"

pip install sentencepiece

# (torch 2.6.0+cu126 requires triton==3.2.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.1.0 which is incompatible.)

get some data:

In [1]:
!export CUDA_VISIBLE_DEVICES=1

In [ ]:
# !wget https://huggingface.co/b74444/style1/resolve/main/base.zip
# !unzip /local/yada/apps/SimpleTuner-a/notebooks/base.zip
# !wget https://huggingface.co/b74444/style1/resolve/main/dark%20and%20bright.zip
# !unzip "dark and bright.zip"

get tagging:

In [1]:
from procslib import get_model, get_model_keys
model = get_model("wdv3_tagger_timm")

Failed to load anime_aesthetic.AnimeAestheticInference: No module named 'pytorch_lightning'
Using global temporary directory: /tmp/unibox_temp
Failed to load complexity_ic9600.IC9600Inference: No module named 'pytorch_lightning'
Failed to load cv2_wrapper.OpenCVMetricsInference: No module named 'skimage'
Failed to load pixiv_compound_score.PixivCompoundScoreInference: No module named 'pytorch_lightning'
Failed to load siglip_aesthetics.SiglipAestheticInference: No module named 'aesthetic_predictor_v2_5'


In [ ]:
import unibox as ub
imgs = ub.ls("/local/yada/apps/SimpleTuner-a/data/b74444", ub.IMG_FILES)

tags = model.infer_many(imgs)

2025-04-13 20:06:45 [INFO] ls: Listing contents of /local/yada/apps/SimpleTuner-a/data/b74444


Listing local files: 0files [00:00, ?files/s]

Inferring tags:   0%|          | 0/28 [00:00<?, ?it/s]

In [6]:
import os

PREFIX = "an anime illustration of "

for _, row in tags.iterrows():
    _caption = row["caption"]
    _path = row["path"]

    _caption = PREFIX + _caption

    txt_path = os.path.splitext(_path)[0] + ".txt"
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(_caption)

In [9]:
tags["taglist"].iloc[0]

'2girls, multiple girls, yuri, long hair, blonde hair, kimono, food, japanese clothes, candy apple, black hair, fireworks, short hair, floral print, red eyes, hair ribbon, blue kimono, closed eyes, ribbon, red ribbon, outdoors, hair ornament, blush, print kimono, one side up, imminent kiss, long sleeves, sash, open mouth, obi, yukata, upper body, shared food, flower, hair flower, pink kimono, signature, night, sidelocks, looking at another, sky, feeding, wide sleeves, nishikigi chisato, inoue takina'

training:

see https://github.com/arot-devs/SimpleTuner-a/blob/main/documentation/quickstart/HIDREAM.md


to fix `The expanded size of the tensor (4096) must match the existing size ...`: set "crop" to `true` in `hidream/multidatabackend.json`

To fix "config different from datset": remove the json inside dataset folder: 
- `rm ./data/b74444/aspect_ratio_bucket_indices_dreambooth-subject.json`